# Prompt Engineering 基础教程

## 学习目标
* 理解Prompt设计的核心原则和最佳实践
* 掌握TACOS框架等主流Prompt结构方法
* 学会使用Claude API进行高效的Prompt Engineering
* 能够编写清晰、结构化、可控的Prompt
* 掌握链式思维、XML标签、多样本提示等高级技巧

## 课程时长
⏱️ **45分钟** (符合Chapter 2核心技能掌握要求)

---

## 📚 内容大纲

1. **前言** (3分钟) - Prompt Engineering的重要性
2. **核心概念** (10分钟) - 基础原理与框架
3. **TACOS框架实践** (15分钟) - 结构化Prompt设计
4. **高级技巧** (12分钟) - Chain-of-Thought、XML标签等
5. **Claude API代码演示** (8分钟) - 实际应用示例
6. **总结与最佳实践** (3分钟) - 关键要点回顾

---

> ⭐ **重要提示**: 本教程专门针对Claude API优化，所有代码示例均可直接运行。请确保已配置好API密钥和代理设置。


## 1. 前言：为什么Prompt Engineering至关重要？(3分钟)

### 什么是Prompt Engineering？

Prompt Engineering是设计和优化与AI模型交互指令的艺术与科学。它不仅仅是写一个问题，而是：

- **精确传达意图** - 让AI理解你真正想要的结果
- **结构化输入** - 通过明确的格式提升输出质量  
- **控制输出行为** - 引导AI按照期望的方式响应
- **提升一致性** - 确保每次交互都能获得可靠的结果

### 为什么对AI Agent开发至关重要？

在AI Agent开发中，Prompt Engineering是核心基础技能，因为：

1. **决定Agent行为质量** - 好的Prompt让Agent更智能、更可靠
2. **提升用户体验** - 精确的指令带来更好的交互体验
3. **降低开发成本** - 比fine-tuning更快速、经济的优化方法
4. **增强可控性** - 通过Prompt调整Agent行为比修改模型更灵活

### Claude vs ChatGPT的Prompt差异

根据最新研究，Claude模型有独特的特点：

- **更严格的指令遵循** - Claude 4系列更字面化地执行指令
- **结构化输入偏好** - 对XML标签、明确分段有更好响应
- **长上下文优势** - 支持更复杂的多步骤推理任务
- **需要更明确的指导** - 不会像GPT那样"猜测"用户意图

> 💡 **关键洞察**: Claude不会填补prompt中的空白，需要用户提供完整、明确的指令。这既是挑战也是优势 - 让你对AI行为有更强的控制力。


## 2. 核心概念：Prompt设计的基本原则 (10分钟)

### 2.1 明确性与具体性原则

**Claude 4的关键变化**: 相比之前的模型，Claude 4更严格地遵循字面指令，不会推测隐含意图。

#### ❌ 模糊的Prompt:
```
分析这个数据集
```

#### ✅ 明确的Prompt:
```
作为数据分析专家，请分析附件中的销售数据集，重点关注：
1. 月度销售趋势
2. 产品类别表现
3. 地区销售差异
请生成包含图表描述的500字分析报告。
```

### 2.2 结构化设计原则

#### 推荐的Prompt结构顺序：
1. **角色定义** (Role) - "你是..."
2. **任务描述** (Task) - "请..."  
3. **背景上下文** (Context) - "基于..."
4. **输出格式** (Output) - "以...格式..."
5. **风格要求** (Style) - "语调..."

### 2.3 TACOS框架简介

TACOS是目前最主流的Prompt结构化框架：

- **T**ask (任务) - 你要模型做什么？
- **A**udience (受众) - 输出面向谁？  
- **C**ontext (上下文) - 补充背景信息
- **O**utput (输出) - 期望的格式/风格
- **S**tyle (风格) - 正式/幽默/学术等

### 2.4 长上下文处理技巧

Claude的长上下文窗口(200K+ tokens)带来新机会：

- **数据前置**: 将长文档放在prompt开头，指令放在结尾
- **XML结构化**: 使用`<document>`, `<instructions>`等标签
- **引用优先**: 要求Claude先提取相关引文，再进行分析

### 2.5 常见错误避免

1. **避免含糊指令** - 具体说明期望的行为
2. **避免矛盾指令** - 检查不同部分的一致性  
3. **避免过度复杂** - 将复杂任务分解为多个步骤
4. **避免省略上下文** - 提供足够的背景信息

> 📖 **学习要点**: 良好的Prompt设计遵循"明确性、结构性、一致性"三大原则，这将在后续的实践中得到验证。


In [1]:
# 环境配置 - 安装必要的依赖包
%pip install anthropic python-dotenv httpx --quiet

import anthropic
import os
import httpx
from dotenv import load_dotenv
import json


Note: you may need to restart the kernel to use updated packages.


In [6]:
# Claude API配置
load_dotenv()  # 加载环境变量

# 配置Claude客户端（包含代理设置）
client = anthropic.Anthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    http_client=httpx.Client(proxy="http://127.0.0.1:7890/")  # 配置代理
)

# 定义辅助函数用于调用Claude API（流式输出）
def call_claude(prompt, system="", max_tokens=1000, temperature=0.7):
    """
    调用Claude API的辅助函数（流式输出）
    
    参数:
    - prompt: 用户输入的提示词
    - system: 系统提示词
    - max_tokens: 最大token数
    - temperature: 随机性参数（0-1，越高越随机）
    """
    full_response = ""
    
    with client.messages.stream(
        model="claude-4-sonnet-20250514",
        max_tokens=max_tokens,
        temperature=temperature,
        system=system,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    ) as stream:
        for text in stream.text_stream:
            print(text, end="", flush=True)
            full_response += text
    
    print()  # 添加换行
    return full_response

print("✅ Claude API 配置完成！（已启用最新Claude 4模型和流式输出）")


✅ Claude API 配置完成！（已启用最新Claude 4模型和流式输出）


## 3. TACOS框架实践：结构化Prompt设计 (15分钟)

### 3.1 TACOS框架详解

TACOS框架是构建高质量Prompt的最佳实践，每个元素都有具体的作用：

#### **T**ask (任务) - 明确要做什么
- 使用动作词：分析、总结、翻译、生成、比较等
- 设定具体目标和期望结果
- 避免含糊不清的表达

#### **A**udience (受众) - 明确输出面向谁  
- 技术专家 vs 普通用户
- 决策者 vs 执行者
- 年龄、教育背景、专业领域

#### **C**ontext (上下文) - 提供背景信息
- 相关数据、文档、历史信息  
- 约束条件和限制
- 当前环境和情况

#### **O**utput (输出) - 明确格式要求
- 文本格式：段落、列表、表格、JSON等
- 长度限制：字数、段落数
- 结构要求：标题、章节、引用等

#### **S**tyle (风格) - 确定语调和风格
- 正式 vs 非正式
- 学术 vs 商业 vs 创意
- 简洁 vs 详细

### 3.2 实际示例对比

让我们通过一个商业分析任务来对比TACOS框架的效果：


In [3]:
# 示例1: 没有结构的Prompt
prompt_bad = "分析一下我们公司的销售情况"

print("❌ 糟糕的Prompt示例:")
print(f"'{prompt_bad}'")
print("\n问题:")
print("- 任务不明确：'分析'具体指什么？")
print("- 缺少上下文：哪个时间段？哪些产品？")
print("- 输出格式未定义：要报告还是数据？")
print("- 受众不明：给谁看的？")
print("- 风格未确定：正式还是内部讨论？")


❌ 糟糕的Prompt示例:
'分析一下我们公司的销售情况'

问题:
- 任务不明确：'分析'具体指什么？
- 缺少上下文：哪个时间段？哪些产品？
- 输出格式未定义：要报告还是数据？
- 受众不明：给谁看的？
- 风格未确定：正式还是内部讨论？


In [4]:
# 示例2: 使用TACOS框架的高质量Prompt
prompt_good = """
**Task**: 作为高级商业分析师，分析我们公司2024年Q3的销售绩效，识别关键趋势和改进机会。

**Audience**: 面向董事会和高级管理层的战略决策报告。

**Context**: 
- 分析范围：2024年7-9月销售数据
- 重点产品线：软件产品A、硬件产品B、服务项目C
- 比较基准：同期去年数据和Q2数据
- 市场环境：竞争对手推出新产品，经济环境不确定

**Output**: 生成结构化的PowerPoint演示要点，包括：
1. 执行摘要（3个关键发现）
2. 销售趋势分析（数据图表描述）
3. 产品线表现对比
4. 风险识别与机会分析
5. 具体行动建议（优先级排序）

**Style**: 
- 语调：专业、客观、数据驱动
- 长度：每部分150-200字
- 格式：简洁的要点形式，易于制作PPT
"""

print("✅ 优秀的TACOS结构化Prompt:")
print(prompt_good)


✅ 优秀的TACOS结构化Prompt:

**Task**: 作为高级商业分析师，分析我们公司2024年Q3的销售绩效，识别关键趋势和改进机会。

**Audience**: 面向董事会和高级管理层的战略决策报告。

**Context**: 
- 分析范围：2024年7-9月销售数据
- 重点产品线：软件产品A、硬件产品B、服务项目C
- 比较基准：同期去年数据和Q2数据
- 市场环境：竞争对手推出新产品，经济环境不确定

**Output**: 生成结构化的PowerPoint演示要点，包括：
1. 执行摘要（3个关键发现）
2. 销售趋势分析（数据图表描述）
3. 产品线表现对比
4. 风险识别与机会分析
5. 具体行动建议（优先级排序）

**Style**: 
- 语调：专业、客观、数据驱动
- 长度：每部分150-200字
- 格式：简洁的要点形式，易于制作PPT



In [7]:
# 实际测试TACOS框架的效果（模拟数据）
sample_sales_data = """
2024年Q3销售数据：
- 软件产品A：营收520万，同比增长15%，环比下降8%
- 硬件产品B：营收380万，同比下降12%，环比增长3%  
- 服务项目C：营收290万，同比增长28%，环比增长15%
- 总营收：1190万，同比增长8%，环比增长2%
- 主要客户：企业客户占70%，个人客户占30%
- 地区分布：北上广深占60%，二线城市占40%
"""

# 组合完整的Prompt（包含实际数据）
full_prompt = f"""
{prompt_good}

**数据输入**:
{sample_sales_data}

请基于以上数据和TACOS框架要求，生成分析报告。
"""

print("🔄 正在调用Claude API进行TACOS框架演示...")
print("⏳ 请稍等，正在生成分析报告...")

# 调用Claude API
response = call_claude(full_prompt, max_tokens=2000, temperature=0.3)
print("\n📊 Claude 基于TACOS框架的分析结果:")
print("="*60)
print(response)


🔄 正在调用Claude API进行TACOS框架演示...
⏳ 请稍等，正在生成分析报告...
# 2024年Q3销售绩效分析报告

## 1. 执行摘要

### 关键发现一：整体增长放缓但保持正向
- Q3总营收1190万元，同比增长8%，环比增长2%
- 增长速度较上半年有所放缓，反映市场竞争加剧影响
- 在经济不确定性背景下，仍实现正增长显示公司韧性

### 关键发现二：产品线表现分化明显
- 服务项目C表现突出，同比增长28%，成为新增长引擎
- 硬件产品B面临挑战，同比下降12%，需重点关注
- 软件产品A增长稳健但环比下滑，存在季节性波动

### 关键发现三：客户结构稳定，地域分布均衡
- 企业客户贡献70%营收，为主要收入来源
- 一二线城市分布6:4，市场覆盖相对均衡
- 客户基础稳固为后续增长提供保障

## 2. 销售趋势分析

### 营收趋势图表描述
**建议图表**：三产品线季度对比柱状图 + 总营收趋势线图
- 软件产品A：呈现"高位震荡"态势，Q3环比回调但同比增长15%保持强劲
- 硬件产品B：显示"触底反弹"迹象，环比转正3%，但同比降幅12%仍需警惕
- 服务项目C：展现"加速上升"趋势，连续两个季度双位数增长

### 同比环比对比分析
**建议图表**：同比环比增长率对比雷达图
- 同比数据显示长期竞争力：软件和服务产品线优势明显
- 环比数据反映短期动能：服务项目势头最强，软件产品需关注下滑原因
- 硬件产品同比环比背离，暗示市场结构性变化

## 3. 产品线表现对比

### 软件产品A（营收占比44%）
- **优势**：市场份额稳固，520万营收贡献最大，同比增长15%显示产品竞争力
- **挑战**：环比下降8%存在季节性或竞争压力，需分析客户流失原因
- **定位**：现金牛产品，需维护市场地位并寻求创新突破

### 硬件产品B（营收占比32%）
- **优势**：环比增长3%显示复苏迹象，产品线调整初见成效
- **挑战**：同比下降12%反映市场竞争激烈，可能受竞争对手新品冲击
- **定位**：问题产品，需重点投入资源进行产品升级和市场策略调整

### 服务项目C（营收占比24%）
- **优势**：双位数高增长（同比28%，环比15%），成为新增长点
- **机会**：市场接受度高，具备进一步

## 4. 高级技巧：Claude特色的Prompt Engineering (12分钟)

### 4.1 XML标签结构化 (Claude 最佳实践)

Claude对XML标签有特殊的优化，能更好地理解和遵循结构化指令：

#### 推荐的XML标签：
- `<instructions>` - 核心指令
- `<context>` - 背景信息  
- `<examples>` - 示例输入输出
- `<constraints>` - 约束条件
- `<output_format>` - 输出格式

### 4.2 Chain-of-Thought (思维链) 技巧

让Claude"想出声"可以显著提升复杂推理任务的表现：

#### 基本模式：
```
请一步步思考这个问题：
1. 首先分析...
2. 然后考虑...
3. 最后得出结论...
```

#### XML增强版本：
```xml
<instructions>
请在<thinking>标签中展示你的思考过程，然后在<answer>标签中给出最终答案。
</instructions>
```

### 4.3 多样本提示 (Few-Shot Prompting)

通过提供高质量示例，引导Claude生成一致的输出格式：

#### 格式模式：
```
以下是几个示例：

示例1:
输入: [示例输入1]
输出: [期望输出1]

示例2:
输入: [示例输入2] 
输出: [期望输出2]

现在请处理：
输入: [实际输入]
输出: 
```

### 4.4 预填充技巧 (Response Prefilling)

通过预设回答的开头，控制Claude的输出格式和风格：

#### 应用场景：
- 确保JSON格式输出
- 统一回答语调
- 引导特定的分析框架

### 4.5 长上下文处理策略

利用Claude 200K+的上下文窗口处理复杂文档：

#### 最佳实践：
1. **文档前置** - 将长文档放在prompt前部
2. **指令后置** - 具体任务指令放在最后
3. **关键信息标记** - 用XML标签突出重要部分
4. **分段处理** - 对超长文档进行逻辑分段


In [8]:
# 高级技巧演示1: XML标签结构化
xml_prompt = """
<context>
你是一名专业的AI产品经理，正在为团队设计AI Agent的核心功能规划。
</context>

<instructions>
基于当前AI Agent发展趋势，设计一个面向企业客户的智能客服Agent的核心功能模块。
</instructions>

<constraints>
- 必须考虑成本效益
- 技术实现难度要适中
- 需要有明确的ROI预期
- 符合数据隐私法规
</constraints>

<output_format>
请按以下JSON格式输出：
{
  "核心功能模块": [
    {
      "模块名称": "",
      "功能描述": "", 
      "技术难度": "低/中/高",
      "预期ROI": "",
      "实现优先级": "P0/P1/P2"
    }
  ],
  "整体架构建议": "",
  "风险评估": ""
}
</output_format>

<examples>
参考但不限于：自然语言处理、知识库检索、多轮对话管理、情感分析等模块。
</examples>
"""

print("🔧 XML结构化Prompt演示:")
print("="*50)
print(xml_prompt)
print("\n🔄 正在调用Claude API...")

# 调用API
xml_response = call_claude(xml_prompt, max_tokens=1500, temperature=0.5)
print("\n📋 Claude基于XML结构的回应:")
print("="*50)
print(xml_response)


🔧 XML结构化Prompt演示:

<context>
你是一名专业的AI产品经理，正在为团队设计AI Agent的核心功能规划。
</context>

<instructions>
基于当前AI Agent发展趋势，设计一个面向企业客户的智能客服Agent的核心功能模块。
</instructions>

<constraints>
- 必须考虑成本效益
- 技术实现难度要适中
- 需要有明确的ROI预期
- 符合数据隐私法规
</constraints>

<output_format>
请按以下JSON格式输出：
{
  "核心功能模块": [
    {
      "模块名称": "",
      "功能描述": "", 
      "技术难度": "低/中/高",
      "预期ROI": "",
      "实现优先级": "P0/P1/P2"
    }
  ],
  "整体架构建议": "",
  "风险评估": ""
}
</output_format>

<examples>
参考但不限于：自然语言处理、知识库检索、多轮对话管理、情感分析等模块。
</examples>


🔄 正在调用Claude API...
```json
{
  "核心功能模块": [
    {
      "模块名称": "智能意图识别与分类",
      "功能描述": "基于NLP技术自动识别客户咨询意图，将问题分类路由到相应处理流程，支持多语言识别",
      "技术难度": "中",
      "预期ROI": "降低人工分流成本60%，提升响应速度3倍",
      "实现优先级": "P0"
    },
    {
      "模块名称": "动态知识库检索",
      "功能描述": "基于向量数据库的语义检索，支持实时知识更新，提供准确率评分和置信度反馈",
      "技术难度": "中",
      "预期ROI": "解决80%常见问题，减少人工客服工作量50%",
      "实现优先级": "P0"
    },
    {
      "模块名称": "多轮对话管理",
      "功能描述": "维护对话上下文，支持复杂业务流程引导，具备话题切换和回溯能力",
      "技术难度": "中",

In [ ]:
# 高级技巧演示2: Chain-of-Thought (思维链)
cot_prompt = """
<instructions>
你是一名AI安全专家，需要评估一个AI Agent在金融服务场景中的潜在风险。

请使用思维链方法，在<thinking>标签中展示你的分析过程，然后在<answer>标签中给出最终的风险评估报告。
</instructions>

<context>
某银行计划部署一个AI Agent用于：
1. 客户咨询回答（包括账户查询、产品介绍）
2. 初步的投资建议生成
3. 风险评估和合规检查提醒
4. 异常交易模式识别

该Agent将：
- 接入客户个人信息数据库
- 连接实时交易系统
- 使用第三方大语言模型API
- 24/7无人值守运行
</context>

<output_format>
<thinking>
[在此展示你的逐步分析过程]
</thinking>

<answer>
风险评估报告：
1. 高风险项：
2. 中风险项：
3. 低风险项：
4. 建议的风险缓解策略：
5. 优先级建议：
</answer>
</output_format>
"""

print("🧠 Chain-of-Thought演示:")
print("="*50)
print(cot_prompt)
print("\n🔄 正在调用Claude API进行思维链分析...")

# 调用API（使用较低的temperature确保分析严谨）
cot_response = call_claude(cot_prompt, max_tokens=2000, temperature=0.2)
print("\n🎯 Claude的思维链分析:")
print("="*50)
print(cot_response)


In [ ]:
# 高级技巧演示3: Few-Shot Prompting (多样本提示)
few_shot_prompt = """
我需要你帮助我生成AI Agent功能需求的标准化描述。请参考以下格式：

<examples>
示例1:
功能名称: 智能邮件分类
输入描述: 接收用户邮箱中的未读邮件
处理逻辑: 基于邮件内容、发送者、主题等信息进行分类
输出结果: 将邮件标记为"工作"、"个人"、"营销"、"垃圾邮件"四类
成功指标: 分类准确率>90%，处理速度<2秒/封

示例2:  
功能名称: 会议纪要生成
输入描述: 会议录音文件和参会人员名单
处理逻辑: 语音转文字后提取关键讨论点、决策和行动项
输出结果: 结构化的会议纪要文档，包含议题、决策、待办事项
成功指标: 关键信息提取完整度>95%，生成时间<会议时长的10%
</examples>

现在请为以下AI Agent功能生成标准化描述：

功能概述: 一个能够自动回复客户咨询的智能客服Agent，主要处理售前产品咨询、订单查询、售后支持等场景。

请按照上述格式生成详细的功能需求描述。
"""

print("📝 Few-Shot Prompting演示:")
print("="*50)
print(few_shot_prompt)
print("\n🔄 正在调用Claude API...")

# 调用API
few_shot_response = call_claude(few_shot_prompt, max_tokens=1000, temperature=0.4)
print("\n📋 Claude基于示例的输出:")
print("="*50)
print(few_shot_response)


## 5. Prompt迭代优化实战 (8分钟)

### 5.1 三阶段迭代法

优秀的Prompt不是一次性设计出来的，需要系统性的迭代优化：

#### 🔄 第一阶段：基础版本
- 明确基本需求和输出格式
- 测试基础功能是否正常
- 识别明显的问题和改进点

#### 🔧 第二阶段：结构优化  
- 应用TACOS框架重构
- 添加必要的约束和示例
- 优化输出格式和质量控制

#### ⚡ 第三阶段：精细调优
- 调整temperature等参数
- 增加边界情况处理
- 优化性能和一致性

### 5.2 常见问题诊断

| 问题现象 | 可能原因 | 解决方案 |
|---------|---------|---------|
| 输出格式不一致 | 格式要求不够明确 | 使用XML标签或Few-shot示例 |
| 回答过于简短 | 缺少详细要求 | 明确指定字数或结构要求 |
| 偏离主题 | 上下文信息混乱 | 重新整理Context部分 |
| 创意不足 | Temperature过低 | 提高随机性参数 |
| 不够专业 | 角色定义不清 | 强化Expert persona |

### 5.3 A/B测试方法

对于关键的Production Prompt，建议进行A/B测试：

#### 测试维度：
- **输出质量** - 准确性、完整性、相关性
- **用户体验** - 可读性、实用性、满意度  
- **技术指标** - 响应时间、Token消耗、稳定性
- **业务指标** - 转化率、用户留存、成本效益

### 5.4 版本管理最佳实践

#### Prompt版本化建议：
```python
PROMPT_VERSION = "v2.1.3"
PROMPT_CHANGELOG = {
    "v2.1.3": "优化XML标签结构，提升JSON输出稳定性",
    "v2.1.2": "增加边界情况处理，修复长文本截断问题", 
    "v2.1.1": "调整temperature参数，平衡创意性和准确性",
    "v2.1.0": "重构TACOS框架，添加Few-shot示例"
}
```


In [9]:
# 迭代优化实战演示
print("🔄 Prompt迭代优化实战演示")
print("="*60)

# 第一阶段：基础版本（V1.0）
prompt_v1 = "帮我写一个AI Agent的产品介绍"

print("📝 V1.0 - 基础版本:")
print(f"Prompt: {prompt_v1}")

# 第二阶段：应用TACOS框架（V2.0）
prompt_v2 = """
**Task**: 作为产品市场经理，撰写一个企业级AI Agent产品的市场介绍材料

**Audience**: 面向中大型企业的CTO和IT决策者

**Context**: 
- 产品定位：企业内部效率提升工具
- 主要竞品：Microsoft Copilot、Google Duet AI
- 差异化优势：更好的私有化部署能力和定制化程度

**Output**: 生成包含以下结构的产品介绍：
1. 产品概述（50字）
2. 核心价值主张（3个要点）
3. 技术优势（3个特性）
4. 典型应用场景（2个案例）
5. 部署方案（简要说明）

**Style**: 专业、简洁、技术导向，避免过度营销语言
"""

print(f"\n📋 V2.0 - TACOS框架版本:")
print(prompt_v2)

# 第三阶段：XML优化版本（V3.0）
prompt_v3 = """
<context>
你是一名专业的B2B产品市场经理，专门负责AI/ML产品的市场推广。目标受众是中大型企业的技术决策者。
</context>

<instructions>
撰写一份企业级AI Agent产品的专业介绍材料，突出技术实力和商业价值。
</instructions>

<background>
产品信息：
- 名称：Enterprise AI Agent Platform
- 定位：企业内部智能化效率提升平台
- 核心差异：私有化部署 + 深度定制能力
- 竞品对比：比Microsoft Copilot更灵活，比开源方案更稳定
</background>

<output_format>
请按以下结构输出：

## 产品概述
[50字精炼描述]

## 核心价值主张
1. [价值点1] - [具体说明]
2. [价值点2] - [具体说明]  
3. [价值点3] - [具体说明]

## 技术优势
### [技术特性1]
[详细说明及业务影响]

### [技术特性2]  
[详细说明及业务影响]

### [技术特性3]
[详细说明及业务影响]

## 典型应用场景
### 场景1: [场景名称]
- 应用描述：
- 效果量化：

### 场景2: [场景名称]
- 应用描述：
- 效果量化：

## 部署方案
[私有化部署的技术要点和实施建议]
</output_format>

<constraints>
- 避免过度营销化的语言
- 每个技术优势都要有具体的业务价值体现
- 应用场景必须包含可量化的效果预期
- 总字数控制在800-1000字
</constraints>
"""

print(f"\n🎯 V3.0 - XML结构化优化版本:")
print(prompt_v3)

print(f"\n🔄 正在测试V3.0版本效果...")
v3_response = call_claude(prompt_v3, max_tokens=1500, temperature=0.4)
print(f"\n📊 V3.0版本输出结果:")
print("="*50)
print(v3_response)


🔄 Prompt迭代优化实战演示
📝 V1.0 - 基础版本:
Prompt: 帮我写一个AI Agent的产品介绍

📋 V2.0 - TACOS框架版本:

**Task**: 作为产品市场经理，撰写一个企业级AI Agent产品的市场介绍材料

**Audience**: 面向中大型企业的CTO和IT决策者

**Context**: 
- 产品定位：企业内部效率提升工具
- 主要竞品：Microsoft Copilot、Google Duet AI
- 差异化优势：更好的私有化部署能力和定制化程度

**Output**: 生成包含以下结构的产品介绍：
1. 产品概述（50字）
2. 核心价值主张（3个要点）
3. 技术优势（3个特性）
4. 典型应用场景（2个案例）
5. 部署方案（简要说明）

**Style**: 专业、简洁、技术导向，避免过度营销语言


🎯 V3.0 - XML结构化优化版本:

<context>
你是一名专业的B2B产品市场经理，专门负责AI/ML产品的市场推广。目标受众是中大型企业的技术决策者。
</context>

<instructions>
撰写一份企业级AI Agent产品的专业介绍材料，突出技术实力和商业价值。
</instructions>

<background>
产品信息：
- 名称：Enterprise AI Agent Platform
- 定位：企业内部智能化效率提升平台
- 核心差异：私有化部署 + 深度定制能力
- 竞品对比：比Microsoft Copilot更灵活，比开源方案更稳定
</background>

<output_format>
请按以下结构输出：

## 产品概述
[50字精炼描述]

## 核心价值主张
1. [价值点1] - [具体说明]
2. [价值点2] - [具体说明]  
3. [价值点3] - [具体说明]

## 技术优势
### [技术特性1]
[详细说明及业务影响]

### [技术特性2]  
[详细说明及业务影响]

### [技术特性3]
[详细说明及业务影响]

## 典型应用场景
### 场景1: [场景名称]
- 应用描述：
- 效果量化：

### 场景2: [场景名称]
- 应用描述：
- 效果量化：

## 部署方案
[私有化部署的技术要点

## 6. 总结与最佳实践 (3分钟)

### 🎯 关键要点回顾

#### 1. Prompt Engineering的核心原则
- **明确性胜过简洁性** - 详细的指令比简短的提示更有效
- **结构化是关键** - TACOS框架提供了可靠的组织方法
- **迭代优化必不可少** - 好的Prompt需要持续改进

#### 2. Claude特色技巧
- **XML标签优势** - Claude对结构化标签有特殊优化
- **思维链威力** - 让AI"想出声"显著提升复杂任务表现
- **长上下文能力** - 200K+窗口为复杂任务提供了新可能

#### 3. 实战应用建议
- **从简单开始** - 先确保基础功能正常，再逐步优化
- **系统性测试** - 建立标准的评估和迭代流程
- **版本化管理** - 像管理代码一样管理Prompt

### 🛠️ 实用工具清单

#### 必备的Prompt工程工具：
```python
# 1. API调用封装函数
def call_claude_with_retry(prompt, max_retries=3):
    # 包含重试逻辑和错误处理
    pass

# 2. Prompt模板库
PROMPT_TEMPLATES = {
    "analysis": "TACOS框架分析模板",
    "generation": "内容生成模板", 
    "classification": "分类任务模板"
}

# 3. 质量评估函数
def evaluate_prompt_quality(prompt, test_cases):
    # 自动化评估prompt表现
    pass
```

### 🚀 下一步学习路径

#### 进阶主题建议：
1. **Multi-Agent协作** - 多个AI Agent的协同设计
2. **Tool Integration** - Prompt + 外部工具集成
3. **Fine-tuning结合** - Prompt Engineering + 模型微调
4. **生产环境部署** - 监控、A/B测试、性能优化

#### 推荐实践项目：
- 设计一个智能客服Agent的完整Prompt体系
- 构建代码审查Agent的多轮对话流程
- 开发数据分析Agent的自动化报告生成

### 💡 最后的建议

> ⭐ **记住**: Prompt Engineering是一门实践性很强的技能。理论知识重要，但真正的提升来自大量的实际练习和迭代优化。

在AI Agent开发中，优秀的Prompt设计能够：
- **降低开发成本** - 避免复杂的模型训练
- **提升用户体验** - 更准确、更可控的AI行为
- **增强商业价值** - 更可靠的AI应用带来更好的ROI

---

## 🎓 课程总结

今天我们学习了：
✅ Prompt Engineering的核心概念和重要性  
✅ TACOS框架的系统化应用  
✅ Claude特色的高级技巧（XML、CoT、Few-shot）  
✅ 三阶段迭代优化方法  
✅ 实际代码演示和最佳实践

**下一节预告**: 《Tool集成实战》- 学习如何让AI Agent调用外部工具和API，实现更强大的功能。
